In [ ]:
import torch
import torchvision.transforms.functional as TF

from PIL import Image
from taesd.taesd import TAESD

dev = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print("Using device", dev)
taesd = TAESD(encoder_path="taesd/models/taesd3_encoder.pth", decoder_path="taesd/models/taesd3_decoder.pth").eval().to(dev)

In [ ]:
imh = 128
imw = imh

im = Image.open(r"00000000.jpg").convert("RGB")
im = TF.to_tensor(im.resize((imw, imh))).unsqueeze(0).to(dev)

im_enc = taesd.encoder(im)
shape = im_enc[0].shape

# print((im_enc - im_enc.to(dtype=torch.float16).to(dtype=torch.float32)).abs().mean().item())
# print(im_enc.element_size() * im_enc.nelement() * 5683 // 1024 // 1024)

l_dim = 128
g_dim = 256
d_dim = 256
o_dim = torch.prod(torch.tensor(shape))

print(f"image shape  : {im.shape[1:]}")
print(f"encoded shape: {shape}")
print(f"num out dims : {o_dim}")

TF.to_pil_image(taesd.decoder(im_enc).clamp(0, 1)[0])

In [ ]:
imh = 128
imw = imh
im_data = []

from tqdm import tqdm
from pathlib import Path

with torch.no_grad():
    imgs = list(Path(r"...").iterdir())
    for im in tqdm(imgs):
        im = Image.open(im).convert("RGB")
        im = TF.to_tensor(im.resize((imw, imh))).unsqueeze(0).to(dev)

        im_enc = taesd.encoder(im)
        im_data.append(im_enc)

In [ ]:
# torch.concatenate(im_data).shape
# TF.to_pil_image(taesd.decoder(im_enc).clamp(0, 1)[0])

In [ ]:
torch.save(torch.concatenate(im_data), "data.pt")

In [ ]:
# import cv2
# import numpy as np

# from tqdm import tqdm
# from pathlib import Path

# height = 128
# batch_size = 256
# paths = list(Path(r"").iterdir())

# j = 0
# frames  = []
# im_data = []

# with torch.no_grad():
#     for i in tqdm(paths):
#         cap = cv2.VideoCapture(str(i))

#         frame_rate   = cap.get(cv2.CAP_PROP_FPS)
#         start_offset = int(11 * frame_rate)
#         end_offset   = int(7  * frame_rate)
#         frame_skips  = int(frame_rate // 6)
#         end_frame    = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - end_offset

#         frame_number = start_offset
#         cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

#         while True:
#             if frame_number > end_frame: break
#             ret, frame = cap.read()
#             if not ret: break
#             frame_number += 1

#             frame = cv2.resize(frame, (height * frame.shape[1]//frame.shape[0], height))
#             frame = TF.to_tensor(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
#             frames.append(frame)

#             j += 1
#             if j % batch_size == 0:
#                 input_  = torch.stack(frames)[:, :, 0:128, 50:178].to(dev)
#                 im_data.append(taesd.encoder(input_))
#                 j = 0
#                 frames = []

#             for k in range(frame_skips):
#                 ret, frame = cap.read()
#                 if not ret: break
#                 frame_number += 1

#         cap.release()

In [ ]:
# im_data.shape
# torch.concatenate(im_data).shape

In [ ]:
# torch.save(torch.concatenate(im_data), "data.pt")